In [ ]:
import cv2
import numpy as np
from scipy.spatial import KDTree
from matplotlib import pyplot as plt
import json
import os

class StarDetector:
    def __init__(self, image_path, center_threshold=30, min_star_area=5):
        """
        初始化星星检测器
        
        Args:
            image_path: 星空图片路径
            center_threshold: 检测中心的阈值
            min_star_area: 最小星星面积
        """
        self.image_path = image_path
        self.center_threshold = center_threshold
        self.min_star_area = min_star_area
        self.star_positions = []
        self.star_intensities = []
        
    def load_and_preprocess_image(self):
        """加载并预处理图片"""
        print(f"正在加载图片: {self.image_path}")
        
        # 对于大图片，我们可以分块读取或使用低分辨率版本
        img = cv2.imread(self.image_path, cv2.IMREAD_GRAYSCALE)
        
        if img is None:
            # 如果直接读取失败，尝试使用imreadmulti处理大图
            try:
                img = cv2.imread(self.image_path, cv2.IMREAD_REDUCED_GRAYSCALE_4)
            except:
                raise ValueError(f"无法读取图片: {self.image_path}")
        
        print(f"图片尺寸: {img.shape}")
        
        # 增强对比度
        img_enhanced = cv2.equalizeHist(img)
        
        # 应用高斯模糊减少噪声
        img_blur = cv2.GaussianBlur(img_enhanced, (3, 3), 0)
        
        return img, img_blur
    
    def detect_stars(self):
        """检测星星位置"""
        img, img_blur = self.load_and_preprocess_image()
        
        # 获取图片尺寸
        height, width = img.shape
        
        # 计算中心点
        center_x, center_y = width // 2, height // 2
        
        # 使用自适应阈值检测星星
        binary = cv2.adaptiveThreshold(
            img_blur, 
            255, 
            cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
            cv2.THRESH_BINARY, 
            11, 
            2
        )
        
        # 寻找轮廓
        contours, _ = cv2.findContours(
            binary, 
            cv2.RETR_EXTERNAL, 
            cv2.CHAIN_APPROX_SIMPLE
        )
        
        self.star_positions = []
        self.star_intensities = []
        
        for contour in contours:
            area = cv2.contourArea(contour)
            if area > self.min_star_area:
                # 计算轮廓中心
                M = cv2.moments(contour)
                if M["m00"] != 0:
                    cX = int(M["m10"] / M["m00"])
                    cY = int(M["m01"] / M["m00"])
                    
                    # 转换为以图片中心为原点的坐标
                    rel_x = cX - center_x
                    rel_y = center_y - cY  # 注意：y轴方向翻转
                    
                    # 获取亮度
                    intensity = float(img[cY, cX])
                    
                    self.star_positions.append((rel_x, rel_y))
                    self.star_intensities.append(intensity)
        
        print(f"检测到 {len(self.star_positions)} 颗星星")
        return self.star_positions, self.star_intensities
    
    def save_positions(self, output_file="star_positions.json"):
        """保存星星位置到JSON文件"""
        data = {
            "image_size": self.load_and_preprocess_image()[0].shape[:2],
            "stars": [
                {
                    "x": float(pos[0]),
                    "y": float(pos[1]),
                    "intensity": float(self.star_intensities[i])
                }
                for i, pos in enumerate(self.star_positions)
            ]
        }
        
        with open(output_file, 'w') as f:
            json.dump(data, f, indent=2)
        
        print(f"星星位置已保存到 {output_file}")
        return output_file
    
    def visualize_detection(self, save_path="star_detection.png"):
        """可视化检测结果"""
        img, _ = self.load_and_preprocess_image()
        img_color = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
        
        height, width = img.shape
        center_x, center_y = width // 2, height // 2
        
        # 绘制检测到的星星
        for pos in self.star_positions:
            px = int(pos[0] + center_x)
            py = int(center_y - pos[1])
            cv2.circle(img_color, (px, py), 2, (0, 255, 0), -1)
        
        # 绘制坐标轴
        cv2.line(img_color, (center_x, 0), (center_x, height), (255, 0, 0), 1)
        cv2.line(img_color, (0, center_y), (width, center_y), (255, 0, 0), 1)
        
        # 保存结果
        cv2.imwrite(save_path, img_color)
        print(f"检测结果已保存到 {save_path}")
        
        return img_color

# 使用示例
def detect_stars_from_image(image_path):
    """从图片中检测星星"""
    detector = StarDetector(
        image_path=image_path,
        center_threshold=30,
        min_star_area=3
    )
    
    # 检测星星
    positions, intensities = detector.detect_stars()
    
    # 保存结果
    detector.save_positions()
    
    # 可视化
    detector.visualize_detection()
    
    return positions, intensities

In [ ]:
detect_stars_from_image('/kaggle/input/eso1242/eso1242a.tif')